In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd '/content/drive/MyDrive/ecehw/project'
%cd '/content/drive/MyDrive/MIDS/ECE661/self-supervised'

/content/drive/MyDrive/MIDS/ECE661/self-supervised


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import random
import os, sys


from resnet import ResNet18_pred, MLP, Block

# from tools.dataset import CIFAR10
from torch.utils.data import DataLoader


device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

device: cuda


In [ ]:

BATCH_SIZE = 128

trainset = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size = BATCH_SIZE, shuffle=True, )

testset = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor()),
    batch_size = BATCH_SIZE, shuffle=True, )

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
len(trainset)

391

In [ ]:

def get_color_distortion(s:float=0.5):
    """
    Function from the paper that create color distortion 
    s: float, the strength of color distortion, for CIFAR 10, the paper use 0.5
    """
    color_jitter = transforms.ColorJitter(0.8*s, 0.8*s, 0.8*s, 0.2*s)
    rnd_color_jitter = transforms.RandomApply([color_jitter], p=0.8)
    rnd_gray = transforms.RandomGrayscale(p=0.2)
    color_distort = transforms.Compose([rnd_color_jitter, rnd_gray])
    return color_distort



train_transform = transforms.Compose([
            # make sure we're using PIL instead of tensor when doing other transform 
            transforms.ToPILImage(),
            transforms.RandomCrop(32, 4),
            #transforms.GaussianBlur(23, sigma=(0.1, 2.0)), # CIFAR 10 doesn't use gaussian blur
            #transforms.RandomResizedCrop(size=32,scale=(0.08,0.1),ratio=(0.75,1.33)),
            transforms.RandomHorizontalFlip(p=0.5),
            #get_color_distortion(),
            transforms.ToTensor(),
            # the normalize numbers are from previous assignment
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])

linear_eval_transform = transforms.Compose([
            #transforms.GaussianBlur(23, sigma=(0.1, 2.0)), # CIFAR 10 doesn't use gaussian blur
            transforms.RandomCrop(32, 4),
            #transforms.RandomResizedCrop(size=32,scale=(0.08,0.1),ratio=(0.75,1.33)),
            transforms.RandomHorizontalFlip(p=0.5),
            #get_color_distortion(),
            transforms.ToTensor(),
            # the normalize numbers are from previous assignment
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])

In [ ]:
def train_epoch(net, epoch, criterion, optimizer, current_learning_rate, scheduler, filename):
  LOSSES = []
  OPTIM_LOSS = float('inf')
  for epoch in range(epoch):
      cost = 0
      correct_examples = 0 
      total_examples = 0 
      for data, targets in trainset:           
          data = data.to(device) 
          targets = targets.to(device)
          # pass  into the model 
          outputs = net(data)
          # calculate loss 
          # loss = compute_loss(yhat, 0.5)
          #outputs = nn.Softmax(dim = 1)(outputs)
          loss = criterion(outputs, targets)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          scheduler.step()
          cost += loss.item()

          # output accuracy 
          _, predicted = torch.max(outputs.data, 1)
          correct_examples += (predicted == targets).sum().item()
          total_examples += targets.size(0)
          

      avg_loss = cost / len(trainset)
      LOSSES.append(avg_loss)
      if avg_loss < OPTIM_LOSS:
          OPTIM_LOSS = avg_loss
          torch.save(net.state_dict(), filename)
      print("Epoch: {}, Average Loss {:f}".format(epoch, avg_loss))
      avg_acc = correct_examples / total_examples
      print("Training accuracy: %.4f" % (avg_acc))
  return LOSSES, avg_acc

In [ ]:
def train_epoch_200_epochs(net, epoch, criterion, optimizer, current_learning_rate, scheduler, filename):
  LOSSES = []
  OPTIM_LOSS = float('inf')
  for epoch in range(epoch):
      if epoch == 100 or epoch == 150:
        current_learning_rate = current_learning_rate * 0.1
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
      cost = 0
      correct_examples = 0 
      total_examples = 0 
      for data, targets in trainset:           
          data = data.to(device) 
          targets = targets.to(device)
          # pass  into the model 
          outputs = net(data)
          # calculate loss 
          # loss = compute_loss(yhat, 0.5)
          #outputs = nn.Softmax(dim = 1)(outputs)
          loss = criterion(outputs, targets)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          # scheduler.step()
          cost += loss.item()

          # output accuracy 
          _, predicted = torch.max(outputs.data, 1)
          correct_examples += (predicted == targets).sum().item()
          total_examples += targets.size(0)
          

      avg_loss = cost / len(trainset)
      LOSSES.append(avg_loss)
      if avg_loss < OPTIM_LOSS:
          OPTIM_LOSS = avg_loss
          torch.save(net.state_dict(), filename)
      print("Epoch: {}, Average Loss {:f}".format(epoch, avg_loss))
      avg_acc = correct_examples / total_examples
      print("Training accuracy: %.4f" % (avg_acc))
  return LOSSES, avg_acc

In [ ]:
def train_epoch_10_percent(net, epoch, criterion, optimizer, current_learning_rate, scheduler, filename):
  LOSSES = []
  OPTIM_LOSS = float('inf')
  for epoch in range(epoch):
      cost = 0
      correct_examples = 0 
      total_examples = 0 
      for batch_ind, (data, targets) in enumerate(trainset):
          if batch_ind == 40:
              break         
          data = data.to(device) 
          targets = targets.to(device)
          # pass  into the model 
          outputs = net(data)
          # calculate loss 
          # loss = compute_loss(yhat, 0.5)
          #outputs = nn.Softmax(dim = 1)(outputs)
          loss = criterion(outputs, targets)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          scheduler.step()
          cost += loss.item()

          # output accuracy 
          _, predicted = torch.max(outputs.data, 1)
          correct_examples += (predicted == targets).sum().item()
          total_examples += targets.size(0)
          

      avg_loss = cost / len(trainset)
      LOSSES.append(avg_loss)
      if avg_loss < OPTIM_LOSS:
          OPTIM_LOSS = avg_loss
          torch.save(net.state_dict(), filename)
      print("Epoch: {}, Average Loss {:f}".format(epoch, avg_loss))
      avg_acc = correct_examples / total_examples
      print("Training accuracy: %.4f" % (avg_acc))
  return LOSSES, avg_acc

In [ ]:
def train_epoch_1_percent(net, epoch, criterion, optimizer, current_learning_rate, scheduler, filename):
  LOSSES = []
  OPTIM_LOSS = float('inf')
  for epoch in range(epoch):
      cost = 0
      correct_examples = 0 
      total_examples = 0 
      for batch_ind, (data, targets) in enumerate(trainset):
          if batch_ind == 4:
              break         
          data = data.to(device) 
          targets = targets.to(device)
          # pass  into the model 
          outputs = net(data)
          # calculate loss 
          # loss = compute_loss(yhat, 0.5)
          #outputs = nn.Softmax(dim = 1)(outputs)
          loss = criterion(outputs, targets)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          scheduler.step()
          cost += loss.item()

          # output accuracy 
          _, predicted = torch.max(outputs.data, 1)
          correct_examples += (predicted == targets).sum().item()
          total_examples += targets.size(0)
          

      avg_loss = cost / len(trainset)
      LOSSES.append(avg_loss)
      if avg_loss < OPTIM_LOSS:
          OPTIM_LOSS = avg_loss
          torch.save(net.state_dict(), filename)
      print("Epoch: {}, Average Loss {:f}".format(epoch, avg_loss))
      avg_acc = correct_examples / total_examples
      print("Training accuracy: %.4f" % (avg_acc))
  return LOSSES, avg_acc

In [ ]:
def train_epoch_200_epochs_10_p(net, epoch, criterion, optimizer, current_learning_rate, scheduler, filename):
  LOSSES = []
  OPTIM_LOSS = float('inf')
  for epoch in range(epoch):
      if epoch == 100 or epoch == 150:
        current_learning_rate = current_learning_rate * 0.1
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
      cost = 0
      correct_examples = 0 
      total_examples = 0
      for batch_ind, (data, targets) in enumerate(trainset):
          if batch_ind == 40:
              break            
          data = data.to(device) 
          targets = targets.to(device)
          # pass  into the model 
          outputs = net(data)
          # calculate loss 
          # loss = compute_loss(yhat, 0.5)
          #outputs = nn.Softmax(dim = 1)(outputs)
          loss = criterion(outputs, targets)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          # scheduler.step()
          cost += loss.item()

          # output accuracy 
          _, predicted = torch.max(outputs.data, 1)
          correct_examples += (predicted == targets).sum().item()
          total_examples += targets.size(0)
          

      avg_loss = cost / len(trainset)
      LOSSES.append(avg_loss)
      if avg_loss < OPTIM_LOSS:
          OPTIM_LOSS = avg_loss
          torch.save(net.state_dict(), filename)
      print("Epoch: {}, Average Loss {:f}".format(epoch, avg_loss))
      avg_acc = correct_examples / total_examples
      print("Training accuracy: %.4f" % (avg_acc))
  return LOSSES, avg_acc

In [ ]:
def train_epoch_200_epochs_1_p(net, epoch, criterion, optimizer, current_learning_rate, scheduler, filename):
  LOSSES = []
  OPTIM_LOSS = float('inf')
  for epoch in range(epoch):
      if epoch == 100 or epoch == 150:
        current_learning_rate = current_learning_rate * 0.1
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
      cost = 0
      correct_examples = 0 
      total_examples = 0
      for batch_ind, (data, targets) in enumerate(trainset):
          if batch_ind == 4:
              break            
          data = data.to(device) 
          targets = targets.to(device)
          # pass  into the model 
          outputs = net(data)
          # calculate loss 
          # loss = compute_loss(yhat, 0.5)
          #outputs = nn.Softmax(dim = 1)(outputs)
          loss = criterion(outputs, targets)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          # scheduler.step()
          cost += loss.item()

          # output accuracy 
          _, predicted = torch.max(outputs.data, 1)
          correct_examples += (predicted == targets).sum().item()
          total_examples += targets.size(0)
          

      avg_loss = cost / len(trainset)
      LOSSES.append(avg_loss)
      if avg_loss < OPTIM_LOSS:
          OPTIM_LOSS = avg_loss
          torch.save(net.state_dict(), filename)
      print("Epoch: {}, Average Loss {:f}".format(epoch, avg_loss))
      avg_acc = correct_examples / total_examples
      print("Training accuracy: %.4f" % (avg_acc))
  return LOSSES, avg_acc

In [ ]:
"""
def train_epoch(net, epoch, criterion, optimizer, current_learning_rate, scheduler, filename):
  LOSSES = []
  OPTIM_LOSS = float('inf')
  for epoch in range(epoch):
      cost = 0
      correct_examples = 0 
      total_examples = 0 
      for data, targets in trainset:
          total_tensor = total_tensor.to(device)
          targets = targets.repeat_interleave(2)  
          targets = targets.to(device)
          # pass  into the model 
          outputs = net(total_tensor)
          # calculate loss 
          # loss = compute_loss(yhat, 0.5)
          #outputs = nn.Softmax(dim = 1)(outputs)
          loss = criterion(outputs, targets)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          scheduler.step()
          cost += loss.item()

          # output accuracy 
          _, predicted = torch.max(outputs.data, 1)
          correct_examples += (predicted == targets).sum().item()
          total_examples += targets.size(0)
          

      avg_loss = cost / len(trainset)
      LOSSES.append(avg_loss)
      if avg_loss < OPTIM_LOSS:
          OPTIM_LOSS = avg_loss
          torch.save(net.state_dict(), filename)
      print("Epoch: {}, Average Loss {:f}".format(epoch, avg_loss))
      avg_acc = correct_examples / total_examples
      print("Training accuracy: %.4f" % (avg_acc))
  return LOSSES, avg_acc
  """

In [ ]:
# clear cuda memory cache
import gc
torch.cuda.empty_cache()
gc.collect()

1412

In [ ]:
trainset = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train=True, download=True, transform=linear_eval_transform),
    batch_size = BATCH_SIZE, shuffle=True, )

Files already downloaded and verified


In [ ]:
NET = ResNet18_pred(3, Block)
CRITERION = nn.CrossEntropyLoss()
#CURRENT_LR = 0.0003
#CURRENT_LR = 0.001
CURRENT_LR = 0.1
#OPTIMIZER = optim.Adam(NET.parameters(), lr=CURRENT_LR, weight_decay=1e-4)
# optimizer = optim.SGD(net.parameters(), lr=0.3*(BATCH_SIZE / 256), weight_decay=1e-6)
OPTIMIZER = optim.SGD(NET.parameters(), lr=CURRENT_LR, momentum=0.9, weight_decay=5e-4)
#SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER , step_size=80, gamma=0.1)
SCHEDULER = torch.optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max=200)
NET.to(device)
NET.train()
train_epoch(NET, 50, CRITERION, OPTIMIZER, CURRENT_LR , SCHEDULER, 'test.pt')



Epoch: 0, Average Loss 1.900710
Training accuracy: 0.3245
Epoch: 1, Average Loss 1.388787
Training accuracy: 0.4893
Epoch: 2, Average Loss 1.144791
Training accuracy: 0.5875
Epoch: 3, Average Loss 0.928661
Training accuracy: 0.6691
Epoch: 4, Average Loss 0.791151
Training accuracy: 0.7194
Epoch: 5, Average Loss 0.685247
Training accuracy: 0.7577
Epoch: 6, Average Loss 0.593766
Training accuracy: 0.7898
Epoch: 7, Average Loss 0.516155
Training accuracy: 0.8197
Epoch: 8, Average Loss 0.450885
Training accuracy: 0.8432
Epoch: 9, Average Loss 0.396796
Training accuracy: 0.8627
Epoch: 10, Average Loss 0.355516
Training accuracy: 0.8750
Epoch: 11, Average Loss 0.318358
Training accuracy: 0.8906
Epoch: 12, Average Loss 0.286276
Training accuracy: 0.9020
Epoch: 13, Average Loss 0.258276
Training accuracy: 0.9129
Epoch: 14, Average Loss 0.242215
Training accuracy: 0.9175
Epoch: 15, Average Loss 0.230878
Training accuracy: 0.9214
Epoch: 16, Average Loss 0.216410
Training accuracy: 0.9265
Epoch: 

([1.9007104921828755,
  1.388786952818751,
  1.1447914576591434,
  0.9286606113624085,
  0.7911506825700745,
  0.6852468516668091,
  0.5937656249536578,
  0.5161551413938518,
  0.45088546664056267,
  0.3967959358335456,
  0.35551648855666673,
  0.3183584355789682,
  0.28627607256860077,
  0.25827578506658755,
  0.242215142139922,
  0.23087784405940634,
  0.216409504785181,
  0.20244475140157717,
  0.19307350734596515,
  0.18369787482692457,
  0.17884255112970576,
  0.1671150904053541,
  0.16251146834334143,
  0.16077928510411163,
  0.15262614526902624,
  0.14721898949416853,
  0.14111122828634345,
  0.14356312485854797,
  0.13447915190530707,
  0.13453796235344295,
  0.1349609971351331,
  0.1300956716194101,
  0.12987835499484215,
  0.1271114190635474,
  0.13867399687199947,
  0.12372361155002928,
  0.12128425184208566,
  0.12630545232168702,
  0.12899630344318003,
  0.11973524606927201,
  0.12385040808878743,
  0.11421496804108096,
  0.11332332612017688,
  0.12146663753425374,
  0.111

In [ ]:
NET.eval()

cost = 0
total, correct = 0, 0
with torch.no_grad():
    for data, label in testset:
        data, label = data.to(device), label.to(device)
        y_pred = NET(data)
        # calculate loss 
        loss = F.cross_entropy(y_pred, label)
        cost += loss.item()

        total += len(label)
        y_hat = torch.argmax(y_pred, dim=1)
        correct += torch.sum(torch.eq(y_hat, label)).item()

avg_loss_test = cost / len(testset)
print("The test accuracy is:", correct/total)

The test accuracy is: 0.8123


In [ ]:
NET = ResNet18_pred(3, Block)
CRITERION = nn.CrossEntropyLoss()
#CURRENT_LR = 0.0003
#CURRENT_LR = 0.001
CURRENT_LR = 0.1
#OPTIMIZER = optim.Adam(NET.parameters(), lr=CURRENT_LR, weight_decay=1e-4)
# optimizer = optim.SGD(net.parameters(), lr=0.3*(BATCH_SIZE / 256), weight_decay=1e-6)
OPTIMIZER = optim.SGD(NET.parameters(), lr=CURRENT_LR, momentum=0.9, weight_decay=5e-4)
#SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER , step_size=80, gamma=0.1)
SCHEDULER = torch.optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max=200)
NET.to(device)
NET.train()
train_epoch_10_percent(NET, 50, CRITERION, OPTIMIZER, CURRENT_LR , SCHEDULER, 'test.pt')



Epoch: 0, Average Loss 0.302375
Training accuracy: 0.1836
Epoch: 1, Average Loss 0.203737
Training accuracy: 0.2516
Epoch: 2, Average Loss 0.189344
Training accuracy: 0.3059
Epoch: 3, Average Loss 0.179501
Training accuracy: 0.3408
Epoch: 4, Average Loss 0.172024
Training accuracy: 0.3707
Epoch: 5, Average Loss 0.168445
Training accuracy: 0.3893
Epoch: 6, Average Loss 0.169469
Training accuracy: 0.3773
Epoch: 7, Average Loss 0.170274
Training accuracy: 0.3758
Epoch: 8, Average Loss 0.167580
Training accuracy: 0.3844
Epoch: 9, Average Loss 0.162807
Training accuracy: 0.4078
Epoch: 10, Average Loss 0.157757
Training accuracy: 0.4279
Epoch: 11, Average Loss 0.152965
Training accuracy: 0.4465
Epoch: 12, Average Loss 0.142807
Training accuracy: 0.4820
Epoch: 13, Average Loss 0.135172
Training accuracy: 0.5221
Epoch: 14, Average Loss 0.127058
Training accuracy: 0.5367
Epoch: 15, Average Loss 0.125007
Training accuracy: 0.5484
Epoch: 16, Average Loss 0.127594
Training accuracy: 0.5404
Epoch: 

([0.30237480289186053,
  0.20373696774777855,
  0.18934429941884698,
  0.17950100636543215,
  0.17202385337761297,
  0.16844459964186334,
  0.16946859829261174,
  0.17027405765660278,
  0.1675801865585015,
  0.1628070664222893,
  0.15775749476059622,
  0.15296481850811894,
  0.14280695683511016,
  0.1351716786699222,
  0.1270580608826464,
  0.1250072159730565,
  0.12759434353665014,
  0.13464237539969443,
  0.13752944146275825,
  0.1341426195695882,
  0.1357095323865066,
  0.1279877158991821,
  0.11621077240580488,
  0.10679040723444556,
  0.10006094451450631,
  0.10064958596168577,
  0.1033307172148429,
  0.10604639141760824,
  0.11305741275972722,
  0.11710980618396379,
  0.11106205184746276,
  0.10453143586283145,
  0.09514034175506943,
  0.08434464010741095,
  0.07931350015313424,
  0.0785290466245178,
  0.0803597069457364,
  0.08601844310760498,
  0.09582682628460858,
  0.095416425439098,
  0.09230934902835075,
  0.09272275922243553,
  0.07802665858622403,
  0.07116341590881348,
 

In [ ]:
NET.eval()

cost = 0
total, correct = 0, 0
with torch.no_grad():
    for data, label in testset:
        data, label = data.to(device), label.to(device)
        y_pred = NET(data)
        # calculate loss 
        loss = F.cross_entropy(y_pred, label)
        cost += loss.item()

        total += len(label)
        y_hat = torch.argmax(y_pred, dim=1)
        correct += torch.sum(torch.eq(y_hat, label)).item()

avg_loss_test = cost / len(testset)
print("The test accuracy is:", correct/total)

The test accuracy is: 0.5022


In [ ]:
NET = ResNet18_pred(3, Block)
CRITERION = nn.CrossEntropyLoss()
#CURRENT_LR = 0.0003
#CURRENT_LR = 0.001
CURRENT_LR = 0.1
#OPTIMIZER = optim.Adam(NET.parameters(), lr=CURRENT_LR, weight_decay=1e-4)
# optimizer = optim.SGD(net.parameters(), lr=0.3*(BATCH_SIZE / 256), weight_decay=1e-6)
OPTIMIZER = optim.SGD(NET.parameters(), lr=CURRENT_LR, momentum=0.9, weight_decay=5e-4)
#SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER , step_size=80, gamma=0.1)
SCHEDULER = torch.optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max=200)
NET.to(device)
NET.train()
train_epoch_1_percent(NET, 50, CRITERION, OPTIMIZER, CURRENT_LR , SCHEDULER, 'test.pt')



Epoch: 0, Average Loss 0.040876
Training accuracy: 0.1172
Epoch: 1, Average Loss 0.049863
Training accuracy: 0.1172
Epoch: 2, Average Loss 0.033887
Training accuracy: 0.1211
Epoch: 3, Average Loss 0.031251
Training accuracy: 0.1016
Epoch: 4, Average Loss 0.026879
Training accuracy: 0.1836
Epoch: 5, Average Loss 0.023455
Training accuracy: 0.1777
Epoch: 6, Average Loss 0.027356
Training accuracy: 0.1875
Epoch: 7, Average Loss 0.026833
Training accuracy: 0.1582
Epoch: 8, Average Loss 0.026743
Training accuracy: 0.1855
Epoch: 9, Average Loss 0.023019
Training accuracy: 0.1934
Epoch: 10, Average Loss 0.022011
Training accuracy: 0.2168
Epoch: 11, Average Loss 0.021772
Training accuracy: 0.1836
Epoch: 12, Average Loss 0.021164
Training accuracy: 0.2344
Epoch: 13, Average Loss 0.021082
Training accuracy: 0.2246
Epoch: 14, Average Loss 0.020786
Training accuracy: 0.2402
Epoch: 15, Average Loss 0.021595
Training accuracy: 0.2168
Epoch: 16, Average Loss 0.020902
Training accuracy: 0.2383
Epoch: 

([0.040875604390488256,
  0.049863398227545305,
  0.033886661919791373,
  0.03125099757748187,
  0.02687902279826991,
  0.023454609429439926,
  0.02735624227987226,
  0.02683255251716165,
  0.026743086402678428,
  0.02301853697013367,
  0.022010839808627468,
  0.021771636460443288,
  0.02116375567053285,
  0.021082336335535853,
  0.020786156434842083,
  0.02159465731257368,
  0.02090215896401564,
  0.02055948805016325,
  0.02104844827481243,
  0.01991071633975524,
  0.020235627813412406,
  0.02001088049710559,
  0.019683600996461367,
  0.019627168355390544,
  0.019259024763960974,
  0.018890091220436195,
  0.01952029067232176,
  0.01862892897232719,
  0.019124879251660593,
  0.01887975903728124,
  0.01857249998985349,
  0.018149925924627983,
  0.018423710027923975,
  0.018350542963618207,
  0.01859538634414868,
  0.018254837721510007,
  0.018164366407467582,
  0.01784658432006836,
  0.01735934028235238,
  0.017999796306385714,
  0.01751305745995563,
  0.016679366226391414,
  0.01744826

In [ ]:
NET.eval()

cost = 0
total, correct = 0, 0
with torch.no_grad():
    for data, label in testset:
        data, label = data.to(device), label.to(device)
        y_pred = NET(data)
        # calculate loss 
        loss = F.cross_entropy(y_pred, label)
        cost += loss.item()

        total += len(label)
        y_hat = torch.argmax(y_pred, dim=1)
        correct += torch.sum(torch.eq(y_hat, label)).item()

avg_loss_test = cost / len(testset)
print("The test accuracy is:", correct/total)

The test accuracy is: 0.3716


In [ ]:
NET = ResNet18_pred(3, Block)
CRITERION = nn.CrossEntropyLoss()
#CURRENT_LR = 0.0003
#CURRENT_LR = 0.001
CURRENT_LR = 0.1
#OPTIMIZER = optim.Adam(NET.parameters(), lr=CURRENT_LR, weight_decay=1e-4)
# optimizer = optim.SGD(net.parameters(), lr=0.3*(BATCH_SIZE / 256), weight_decay=1e-6)
OPTIMIZER = optim.SGD(NET.parameters(), lr=CURRENT_LR, momentum=0.9, weight_decay=5e-4)
#SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER , step_size=80, gamma=0.1)
SCHEDULER = torch.optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max=200)
NET.to(device)
NET.train()
train_epoch_200_epochs(NET, 200, CRITERION, OPTIMIZER, CURRENT_LR , SCHEDULER, 'test.pt')


Epoch: 0, Average Loss 1.809095
Training accuracy: 0.3513
Epoch: 1, Average Loss 1.283650
Training accuracy: 0.5324
Epoch: 2, Average Loss 0.983742
Training accuracy: 0.6481
Epoch: 3, Average Loss 0.803510
Training accuracy: 0.7150
Epoch: 4, Average Loss 0.659460
Training accuracy: 0.7686
Epoch: 5, Average Loss 0.539391
Training accuracy: 0.8122
Epoch: 6, Average Loss 0.465035
Training accuracy: 0.8387
Epoch: 7, Average Loss 0.407115
Training accuracy: 0.8582
Epoch: 8, Average Loss 0.363887
Training accuracy: 0.8737
Epoch: 9, Average Loss 0.335131
Training accuracy: 0.8837
Epoch: 10, Average Loss 0.293170
Training accuracy: 0.8989
Epoch: 11, Average Loss 0.282061
Training accuracy: 0.9018
Epoch: 12, Average Loss 0.260139
Training accuracy: 0.9084
Epoch: 13, Average Loss 0.242687
Training accuracy: 0.9156
Epoch: 14, Average Loss 0.232123
Training accuracy: 0.9192
Epoch: 15, Average Loss 0.226547
Training accuracy: 0.9215
Epoch: 16, Average Loss 0.210818
Training accuracy: 0.9267
Epoch: 

KeyboardInterrupt: ignored

In [ ]:
NET = ResNet18_pred(3, Block)
CRITERION = nn.CrossEntropyLoss()
#CURRENT_LR = 0.0003
#CURRENT_LR = 0.001
CURRENT_LR = 0.1
#OPTIMIZER = optim.Adam(NET.parameters(), lr=CURRENT_LR, weight_decay=1e-4)
# optimizer = optim.SGD(net.parameters(), lr=0.3*(BATCH_SIZE / 256), weight_decay=1e-6)
OPTIMIZER = optim.SGD(NET.parameters(), lr=CURRENT_LR, momentum=0.9, weight_decay=5e-4)
#SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER , step_size=80, gamma=0.1)
SCHEDULER = torch.optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max=200)
NET.to(device)
NET.train()
train_epoch_200_epochs_10_p(NET, 200, CRITERION, OPTIMIZER, CURRENT_LR , SCHEDULER, 'test.pt')

Epoch: 0, Average Loss 0.294276
Training accuracy: 0.1781
Epoch: 1, Average Loss 0.202809
Training accuracy: 0.2707
Epoch: 2, Average Loss 0.184640
Training accuracy: 0.3309
Epoch: 3, Average Loss 0.176356
Training accuracy: 0.3748
Epoch: 4, Average Loss 0.168305
Training accuracy: 0.3820
Epoch: 5, Average Loss 0.157866
Training accuracy: 0.4281
Epoch: 6, Average Loss 0.155499
Training accuracy: 0.4359
Epoch: 7, Average Loss 0.150746
Training accuracy: 0.4551
Epoch: 8, Average Loss 0.149297
Training accuracy: 0.4578
Epoch: 9, Average Loss 0.142495
Training accuracy: 0.4811
Epoch: 10, Average Loss 0.138921
Training accuracy: 0.5008
Epoch: 11, Average Loss 0.133629
Training accuracy: 0.5215
Epoch: 12, Average Loss 0.130654
Training accuracy: 0.5371
Epoch: 13, Average Loss 0.122441
Training accuracy: 0.5641
Epoch: 14, Average Loss 0.121094
Training accuracy: 0.5686
Epoch: 15, Average Loss 0.116254
Training accuracy: 0.5984
Epoch: 16, Average Loss 0.113159
Training accuracy: 0.6037
Epoch: 

([0.29427647590637207,
  0.20280922892148537,
  0.18464030146293933,
  0.17635644305392603,
  0.16830468879026525,
  0.15786583435809826,
  0.15549855799321324,
  0.15074598575796921,
  0.1492965187867889,
  0.14249504710097446,
  0.13892085503434282,
  0.13362908881643545,
  0.13065442740154998,
  0.12244057152277368,
  0.12109433537553949,
  0.11625369811606834,
  0.11315936185514835,
  0.11078414496253519,
  0.1017987583299427,
  0.10529745158636966,
  0.1003350371594929,
  0.09857528106026027,
  0.09424312065934282,
  0.0936861612912639,
  0.08905610663201803,
  0.08878464055488176,
  0.0861135792854192,
  0.08214228141033436,
  0.08217821157801791,
  0.075909246721536,
  0.0728964168397362,
  0.07274077432539762,
  0.0722793613553352,
  0.07171355092616948,
  0.0708716389773142,
  0.06477428191458173,
  0.06267975709017586,
  0.06211421899783337,
  0.05982889406516424,
  0.05878436138562839,
  0.05870236726977941,
  0.058892805024486065,
  0.054017079303331696,
  0.053315643078225

In [ ]:
NET.eval()

cost = 0
total, correct = 0, 0
with torch.no_grad():
    for data, label in testset:
        data, label = data.to(device), label.to(device)
        y_pred = NET(data)
        # calculate loss 
        loss = F.cross_entropy(y_pred, label)
        cost += loss.item()

        total += len(label)
        y_hat = torch.argmax(y_pred, dim=1)
        correct += torch.sum(torch.eq(y_hat, label)).item()

avg_loss_test = cost / len(testset)
print("The test accuracy is:", correct/total)

The test accuracy is: 0.8606


In [ ]:
NET = ResNet18_pred(3, Block)
CRITERION = nn.CrossEntropyLoss()
#CURRENT_LR = 0.0003
#CURRENT_LR = 0.001
CURRENT_LR = 0.1
#OPTIMIZER = optim.Adam(NET.parameters(), lr=CURRENT_LR, weight_decay=1e-4)
# optimizer = optim.SGD(net.parameters(), lr=0.3*(BATCH_SIZE / 256), weight_decay=1e-6)
OPTIMIZER = optim.SGD(NET.parameters(), lr=CURRENT_LR, momentum=0.9, weight_decay=5e-4)
#SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER , step_size=80, gamma=0.1)
SCHEDULER = torch.optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max=200)
NET.to(device)
NET.train()
train_epoch_200_epochs_1_p(NET, 200, CRITERION, OPTIMIZER, CURRENT_LR , SCHEDULER, 'test.pt')

Epoch: 0, Average Loss 0.034855
Training accuracy: 0.1289
Epoch: 1, Average Loss 0.044257
Training accuracy: 0.1309
Epoch: 2, Average Loss 0.035510
Training accuracy: 0.1387
Epoch: 3, Average Loss 0.030711
Training accuracy: 0.1816
Epoch: 4, Average Loss 0.025128
Training accuracy: 0.2012
Epoch: 5, Average Loss 0.026560
Training accuracy: 0.1465
Epoch: 6, Average Loss 0.022883
Training accuracy: 0.2227
Epoch: 7, Average Loss 0.022576
Training accuracy: 0.1973
Epoch: 8, Average Loss 0.021773
Training accuracy: 0.2168
Epoch: 9, Average Loss 0.021376
Training accuracy: 0.2070
Epoch: 10, Average Loss 0.021227
Training accuracy: 0.2129
Epoch: 11, Average Loss 0.020524
Training accuracy: 0.2559
Epoch: 12, Average Loss 0.021208
Training accuracy: 0.2051
Epoch: 13, Average Loss 0.020420
Training accuracy: 0.2324
Epoch: 14, Average Loss 0.020765
Training accuracy: 0.2461
Epoch: 15, Average Loss 0.021034
Training accuracy: 0.2363
Epoch: 16, Average Loss 0.020472
Training accuracy: 0.2090
Epoch: 

([0.034854983429774604,
  0.04425675484835339,
  0.035509950974408316,
  0.030711034374773655,
  0.025128341391873178,
  0.026559578488245035,
  0.022883231682545694,
  0.022576261969173655,
  0.021772868797907134,
  0.021375668018370333,
  0.021227213427843644,
  0.02052400423132855,
  0.021208393604249295,
  0.020419705554347516,
  0.020765319199818173,
  0.021033801386118545,
  0.020471833551021487,
  0.021176555882329525,
  0.02016517055004149,
  0.020432790831836592,
  0.01986670158708187,
  0.019792400052785264,
  0.01890321918155836,
  0.01916274908558487,
  0.01902224980961636,
  0.018576084195500445,
  0.01869254404931422,
  0.018192134244972482,
  0.018610048172114146,
  0.018308194092167614,
  0.017936443428859077,
  0.018379262341257863,
  0.01790996189312557,
  0.01698099896121208,
  0.01790550572183126,
  0.017633208533382173,
  0.016891629799552586,
  0.017170138676148243,
  0.016957248873113062,
  0.017670464942522366,
  0.017940806915692966,
  0.017508143049371823,
  0

In [ ]:
NET.eval()

cost = 0
total, correct = 0, 0
with torch.no_grad():
    for data, label in testset:
        data, label = data.to(device), label.to(device)
        y_pred = NET(data)
        # calculate loss 
        loss = F.cross_entropy(y_pred, label)
        cost += loss.item()

        total += len(label)
        y_hat = torch.argmax(y_pred, dim=1)
        correct += torch.sum(torch.eq(y_hat, label)).item()

avg_loss_test = cost / len(testset)
print("The test accuracy is:", correct/total)

The test accuracy is: 0.5789
